# Question 1

In [1]:
import pulp

In [2]:
values = [10, 8, 12, 4, 5, 10, 6, 9, 7, 9]  
weights = [5, 7, 4, 3, 5, 3, 4, 6, 4, 6]   
n = len(values)

In [3]:
def solve_knapsack(max_weight):

    prob = pulp.LpProblem("Knapsack Problem", pulp.LpMaximize)
    
    # list of binary decision variables
    item_vars = [pulp.LpVariable(f"item_{i}", cat="Binary") for i in range(n)]

    # function to maximize total value of the selected items
    prob += pulp.lpSum([values[i] * item_vars[i] for i in range(n)])
    
    # constraint for total weight < max weight
    prob += pulp.lpSum([weights[i] * item_vars[i] for i in range(n)]) <= max_weight
    
    status = prob.solve(pulp.PULP_CBC_CMD(msg=True))
    if status == pulp.LpStatusOptimal:
        selected_items_pulp = [i for i in range(n) if pulp.value(item_vars[i]) == 1]
        total_value_pulp = pulp.value(prob.objective)
        return total_value_pulp, selected_items_pulp
    else:
        return None, []

result_23_kg = solve_knapsack(23)
result_20_kg = solve_knapsack(20)
result_26_kg = solve_knapsack(26)

if result_23_kg:
    print(f"Total value for 23 kg: {result_23_kg[0]}, Items: {result_23_kg[1]}")
if result_20_kg:
    print(f"Total value for 20 kg: {result_20_kg[0]}, Items: {result_20_kg[1]}")
if result_26_kg:
    print(f"Total value for 26 kg: {result_26_kg[0]}, Items: {result_26_kg[1]}")

Total value for 23 kg: 49.0, Items: [0, 2, 3, 5, 6, 8]
Total value for 20 kg: 45.0, Items: [0, 2, 5, 6, 8]
Total value for 26 kg: 54.0, Items: [0, 2, 5, 6, 7, 8]


/opt/anaconda3/lib/python3.8/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


# Question 2

In [4]:
import numpy as np
from scipy.optimize import linprog

### Task 1:
- (a) Which list(s) of places could you recommend to him? This solution will be called ListVisit 1.
- (b) Which list(s) of places could you recommend to him if Doe’s budget is now 65 e, the maximum duration still being 14
hours ?
- (c) Which list(s) of places could you recommend to him if Doe’s budget is now 90 eand the maximum duration being 10 hours?

In [5]:
# Sites and their attributes
sites = ['TE', 'ML', 'AT', 'MO', 'JT', 'CA', 'CP', 'CN', 'BS', 'SC', 'PC', 'TM', 'AC']
durations = np.array([9/2, 3, 1, 2, 3/2, 2, 5/2, 2, 2, 3/2, 3/4, 2, 3/2])  # in hours
prices = np.array([16.5, 14, 10.5, 11, 0, 10, 10, 7, 10, 8.5, 0, 12, 0])  # in euros
distances = np.array([
    [0, 3.8, 2.1, 2.4, 3.5, 4.2, 5.0, 4.4, 5.5, 4.2, 2.5, 3.1, 1.9],
    [3.8, 0, 3.8, 1.1, 1.3, 3.3, 1.3, 1.1, 3.4, 0.8, 1.7, 2.5, 2.8],
    [2.1, 3.8, 0, 3.1, 3.0, 5.8, 4.8, 4.9, 4.3, 4.6, 2.2, 4.4, 1.0],
    [2.4, 1.1, 3.1, 0, 0.9, 3.1, 2.5, 2.0, 3.9, 1.8, 1.0, 2.3, 2.1],
    [3.5, 1.3, 3.0, 0.9, 0, 4.2, 2.0, 2.4, 2.7, 2.0, 1.0, 3.4, 2.1],
    [4.2, 3.3, 5.8, 3.1, 4.2, 0, 2.0, 2.7, 2.7, 2.0, 1.0, 3.4, 2.1],
    [5.0, 1.3, 4.8, 2.5, 2.0, 2.0, 0, 2.5, 6.5, 2.6, 3.8, 1.3, 4.9],
    [4.4, 1.1, 4.9, 2.0, 2.4, 2.7, 2.5, 0, 3.7, 0.9, 2.7, 3.4, 3.8],
    [5.5, 3.4, 4.3, 3.9, 2.7, 2.7, 6.5, 3.7, 0, 4.5, 0.4, 2.7, 3.9],
    [4.2, 0.8, 4.6, 1.8, 2.0, 2.0, 2.6, 0.9, 4.5, 0, 2.8, 2.7, 3.8],
    [2.5, 1.7, 2.2, 1.0, 1.0, 1.0, 3.8, 2.7, 0.4, 2.8, 0, 3.0, 1.2],
    [3.1, 2.5, 4.4, 2.3, 3.4, 3.4, 1.3, 3.4, 2.7, 2.7, 3.0, 0, 3.6],
    [1.9, 2.8, 1.0, 2.1, 2.1, 2.1, 4.9, 3.8, 3.9, 3.8, 1.2, 3.6, 0]
])

# Average walking speed is about 5 km/h; converting distances to walking time in hours
walking_speed = 5
travel_time = np.array(distances) / walking_speed
total_durations = durations + np.min(travel_time + travel_time.T, axis=0) * 2

# Constraints for each scenario
constraints = {
    "ListVisit1": {"budget": 75, "duration": 14},
    "ListVisit2": {"budget": 65, "duration": 14},
    "ListVisit3": {"budget": 90, "duration": 10}
}

results = {}

# Solving each case using linear programming
for name, constraint in constraints.items():
    c = -np.ones(len(sites))  # Objective function to maximize number of sites visited
    A = [total_durations, prices]  # Constraints: total duration including travel, and cost
    b = [constraint["duration"], constraint["budget"]]  # Bounds from each scenario's constraints

    result = linprog(c, A_ub=A, b_ub=b, bounds=[(0, 1) for _ in sites], method='highs')
    visited_sites = np.where(result.x > 0.5)[0]  # Sites where x_i > 0.5 are considered visited
    selected_sites = [sites[i] for i in visited_sites]
    total_duration = np.sum(durations[visited_sites])
    total_cost = np.sum(prices[visited_sites])

    results[name] = (selected_sites, total_duration, total_cost)

# Display the results for each scenario
for scenario, (sites_list, duration, cost) in results.items():
    print(f"{scenario}: {sites_list}, Total Duration: {duration}, Total Cost: {cost}")

ListVisit1: ['AT', 'JT', 'CA', 'CN', 'BS', 'SC', 'PC', 'TM', 'AC'], Total Duration: 14.25, Total Cost: 58.0
ListVisit2: ['AT', 'JT', 'CA', 'CN', 'BS', 'SC', 'PC', 'TM', 'AC'], Total Duration: 14.25, Total Cost: 58.0
ListVisit3: ['AT', 'JT', 'CN', 'BS', 'SC', 'PC', 'AC'], Total Duration: 10.25, Total Cost: 36.0


<b>Observations:</b>
- In ListVisit 1 and ListVisit 2, both with a time constraint of 14 hours, we see a consistent selection of sites, maximizing the number of visits within the budget 75 and 65 euros.
- ListVisit 3, despite having only 10 hours includes high-value sites, making full use of a larger budget but the total budget resulted 36 euros. This shows the algorithm's flexibility in adapting to varying budget and time constraints while still optimizing the visits.

In [6]:
# List of sites
sites = ['TE', 'ML', 'AT', 'MO', 'JT', 'CA', 'CP', 'CN', 'BS', 'SC', 'PC', 'TM', 'AC']

# Durations (in hours)
durations = [9/2, 3, 1, 2, 3/2, 2, 5/2, 2, 2, 3/2, 3/4, 2, 3/2]

# Prices (in euros)
prices = [16.5, 14, 10.5, 11, 0, 10, 10, 7, 10, 8.5, 0, 12, 0]

# Appreciations (from 1 to 5 stars)
appreciations = [5, 4, 3, 2, 3, 4, 1, 5, 4, 1, 3, 2, 5]

# Distances (in kms of walking) between pairs of sites
distances = {
    ("TE", "ML"): 3.8, ("TE", "AT"): 2.1, ("TE", "MO"): 2.4, ("TE", "JT"): 3.5, ("TE", "CA"): 4.2, 
    ("TE", "CP"): 5.0, ("TE", "CN"): 4.4, ("TE", "BS"): 5.5, ("TE", "SC"): 4.2, ("TE", "PC"): 2.5, 
    ("TE", "TM"): 3.1, ("TE", "AC"): 1.9, ("ML", "AT"): 3.8, ("ML", "MO"): 1.1, ("ML", "JT"): 1.3, 
    ("ML", "CA"): 3.3, ("ML", "CP"): 1.3, ("ML", "CN"): 1.1, ("ML", "BS"): 3.4, ("ML", "SC"): 0.8, 
    ("ML", "PC"): 1.7, ("ML", "TM"): 2.5, ("ML", "AC"): 2.8, ("AT", "MO"): 3.1, ("AT", "JT"): 3.0, 
    ("AT", "CA"): 5.8, ("AT", "CP"): 4.8, ("AT", "CN"): 4.9, ("AT", "BS"): 4.3, ("AT", "SC"): 4.6, 
    ("AT", "PC"): 2.2, ("AT", "TM"): 4.4, ("AT", "AC"): 1.0, ("MO", "JT"): 0.9, ("MO", "CA"): 3.1, 
    ("MO", "CP"): 2.5, ("MO", "CN"): 2.0, ("MO", "BS"): 3.9, ("MO", "SC"): 1.8, ("MO", "PC"): 1.0, 
    ("MO", "TM"): 2.3, ("MO", "AC"): 2.1, ("JT", "CA"): 4.2, ("JT", "CP"): 2.0, ("JT", "CN"): 2.4, 
    ("JT", "BS"): 2.7, ("JT", "SC"): 2.0, ("JT", "PC"): 1.0, ("JT", "TM"): 3.4, ("JT", "AC"): 2.1, 
    ("CA", "CP"): 3.5, ("CA", "CN"): 2.7, ("CA", "BS"): 6.5, ("CA", "SC"): 2.6, ("CA", "PC"): 3.8, 
    ("CA", "TM"): 1.3, ("CA", "AC"): 4.9, ("CP", "CN"): 0.85, ("CP", "BS"): 3.7, ("CP", "SC"): 0.9, 
    ("CP", "PC"): 2.7, ("CP", "TM"): 3.4, ("CP", "AC"): 3.8, ("CN", "BS"): 4.5, ("CN", "SC"): 0.4, 
    ("CN", "PC"): 2.8, ("CN", "TM"): 2.7, ("CN", "AC"): 3.9, ("BS", "SC"): 4.2, ("BS", "PC"): 3.3, 
    ("BS", "TM"): 5.7, ("BS", "AC"): 3.8, ("SC", "PC"): 2.5, ("SC", "TM"): 2.6, ("SC", "AC"): 3.6, 
    ("PC", "TM"): 3.0, ("PC", "AC"): 1.2, ("TM", "AC"): 2.1
}

In [7]:
def custom_maximize_visits(budget, hours, prices, durations, sites):
    site_data = sorted(zip(sites, prices, durations), key=lambda x: (x[1], x[2]))

    visited_sites = []
    total_cost = 0
    total_duration = 0

    for site, price, duration in site_data:
        if total_cost + price <= budget and total_duration + duration <= hours:
            visited_sites.append(site)
            total_cost += price
            total_duration += duration
    
    return visited_sites, total_cost, total_duration

# Case (a): Budget = 75 euros, Time = 14 hours
list_visit_1, cost_1, time_1 = custom_maximize_visits(75, 14, prices, durations, sites)

# Case (b): Budget = 65 euros, Time = 14 hours
list_visit_2, cost_2, time_2 = custom_maximize_visits(65, 14, prices, durations, sites)

# Case (c): Budget = 90 euros, Time = 10 hours
list_visit_3, cost_3, time_3 = custom_maximize_visits(90, 10, prices, durations, sites)

print("ListVisit 1:", list_visit_1)
print("Cost:", cost_1, "Total Time:", time_1)

print("ListVisit 2:", list_visit_2)
print("Cost:", cost_2, "Total Time:", time_2)

print("ListVisit 3:", list_visit_3)
print("Cost:", cost_3, "Total Time:", time_3)

ListVisit 1: ['PC', 'JT', 'AC', 'CN', 'SC', 'CA', 'BS', 'CP']
Cost: 45.5 Total Time: 13.75
ListVisit 2: ['PC', 'JT', 'AC', 'CN', 'SC', 'CA', 'BS', 'CP']
Cost: 45.5 Total Time: 13.75
ListVisit 3: ['PC', 'JT', 'AC', 'CN', 'SC', 'CA']
Cost: 25.5 Total Time: 9.25


### Task 2 

##### We keep the budget of **75 euros** and the maximum duration being **14 hours**. Actually, Mr. Doe has some preferences among these tourist sites, and he expresses them as follows:

- **Preference 1**: If two sites are geographically very close (within a radius of 1 km of walking), he will prefer to visit these two sites instead of visiting only one.
- **Preference 2**: He absolutely wants to visit the **Eiffel Tower (TE)** and **Catacombes (CA)**.
- **Preference 3**: If he visits **Avenue des Champs-Elysées (AC)**, then he will not visit the **Sainte-Chapelle (SC)**.
- **Preference 4**: He absolutely wants to visit the **Arc de Triomphe (AT)**.
- **Preference 5**: If he visits the **Louvre Museum (ML)**, then he must visit the **Musée d’Orsay (MO)**.


- **(a)** For each of the five preferences above, suggest to Mr. Doe, one or more lists of tourist sites to visit. Are the obtained lists
different from the solution ListVisit1?

In [11]:
def apply_preferences_individually(budget, hours, prices, durations, sites, distances):
    results = {}

    # Base settings
    base_sites = ['TE', 'CA', 'AT']  # Mandatory for all scenarios due to absolute preferences
    if 'ML' in sites:
        base_sites.append('MO')  # Dependency of ML on MO

    # Preferences application
    preferences = {
        'Preference 1': lambda visited, site: any(distances[sites.index(site)][sites.index(vs)] <= 1.0 for vs in visited),
        'Preference 2': lambda visited, site: site in ['TE', 'CA'],
        'Preference 3': lambda visited, site: not ('AC' in visited and site == 'SC' or 'SC' in visited and site == 'AC'),
        'Preference 4': lambda visited, site: site in ['AT'],
        'Preference 5': lambda visited, site: not ('ML' in visited and 'MO' not in visited)
    }

    # Apply each preference separately
    for pref, condition in preferences.items():
        visited_sites = set(base_sites)
        total_cost = sum(prices[sites.index(site)] for site in visited_sites)
        total_duration = sum(durations[sites.index(site)] for site in visited_sites)

        for site, price, duration in zip(sites, prices, durations):
            if site in visited_sites:
                continue
            if total_cost + price > budget or total_duration + duration > hours:
                continue
            if condition(visited_sites, site):
                visited_sites.add(site)
                total_cost += price
                total_duration += duration

        results[pref] = (visited_sites, total_cost, total_duration)

    return results

def check_lists_similarity(list1, list2):
    return set(list1) == set(list2)
    

list_visit_1 = {'PC', 'JT', 'AC', 'CN', 'SC', 'CA', 'BS', 'CP'}

# Apply preferences individually
individual_preferences = apply_preferences_individually(75, 14, prices, durations, sites, distances_km)

# Output results and check against ListVisit 1
comparison_results = {}
for pref, result in individual_preferences.items():
    visited, cost, duration = result
    comparison_results[pref] = {
        'Visited Sites': visited,
        'Total Cost': cost,
        'Total Time': duration,
        'Matches ListVisit 1': check_lists_similarity(visited, list_visit_1)
    }

# Print results
for pref, data in comparison_results.items():
    print(f"{pref}:")
    print(f"  Visited Sites: {data['Visited Sites']}")
    print(f"  Total Cost: {data['Total Cost']}")
    print(f"  Total Time: {data['Total Time']}")
    print(f"  Matches ListVisit 1: {data['Matches ListVisit 1']}\n")

NameError: name 'distances_km' is not defined

### (b) If Mr. Doe wishes, at the same time, to take into account Preference 1 and Preference 2, which list(s) would you recommend to him?

In [10]:
def optimize_combined_preferences(budget, hours, prices, durations, sites, distances):
    # Start with mandatory sites from Preference 2
    mandatory_sites = {'TE', 'CA'}
    
    visited_sites = set(mandatory_sites)
    total_cost = sum(prices[sites.index(site)] for site in visited_sites)
    total_duration = sum(durations[sites.index(site)] for site in visited_sites)

    # Site data for iteration
    site_data = list(zip(sites, prices, durations))

    # Combine Preference 1: visit geographically close sites
    # with Preference 2: mandatory visits to TE and CA
    for site, price, duration in site_data:
        if site in visited_sites:
            continue  # Skip if already included in the plan
        if total_cost + price > budget or total_duration + duration > hours:
            continue  # Skip if adding this site exceeds budget or time

        # Check for geographic proximity for Preference 1
        site_index = sites.index(site)
        add_site = False
        for added_site in list(visited_sites):
            added_index = sites.index(added_site)
            if distances[site_index][added_index] <= 1.0:  # Within 1 km
                add_site = True
                break

        # Add site if it meets proximity requirement or is part of the mandatory sites
        if add_site:
            visited_sites.add(site)
            total_cost += price
            total_duration += duration

    return visited_sites, total_cost, total_duration

combined_list_visit, combined_cost, combined_time = optimize_combined_preferences(
    75, 14, prices, durations, sites, distances)

print("Recommended Sites:", combined_list_visit)
print("Total Cost:", combined_cost)
print("Total Time:", combined_time)

KeyError: 1

In [12]:
def optimize_preferences_1_and_3(budget, hours, prices, durations, sites, distances):
    visited_sites = set()
    total_cost = 0
    total_duration = 0
    site_data = list(zip(sites, prices, durations))

    for site, price, duration in site_data:
        if total_cost + price > budget or total_duration + duration > hours:
            continue  # Skip if adding this site exceeds budget or time

        # Preference 3: Mutual Exclusivity
        if ('AC' in visited_sites and site == 'SC') or ('SC' in visited_sites and site == 'AC'):
            continue

        # Preference 1: Geographic Proximity
        site_index = sites.index(site)
        add_site = True  # Assume we can add the site unless proven otherwise by lack of proximity
        if visited_sites:  # Check proximity if there are already sites visited
            add_site = any(distances[site_index][sites.index(vs)] <= 1.0 for vs in visited_sites)

        if add_site:
            visited_sites.add(site)
            total_cost += price
            total_duration += duration

    return visited_sites, total_cost, total_duration

combined_list_visit, combined_cost, combined_time = optimize_preferences_1_and_3(
    75, 14, prices, durations, sites, distances_km)

print("Recommended Sites:", combined_list_visit)
print("Total Cost:", combined_cost)
print("Total Time:", combined_time)

Recommended Sites: {'TE'}
Total Cost: 16.5
Total Time: 4.5


### (d) If Mr. Doe wishes, at the same time, to take into account Preference 1 and Preference 4, which list(s) would you recommend to him?

In [14]:
def optimize_preferences_1_and_4(budget, hours, prices, durations, sites, distances):
    # Start with mandatory site from Preference 4
    mandatory_sites = {'AT'}
    
    visited_sites = set(mandatory_sites)
    total_cost = sum(prices[sites.index(site)] for site in visited_sites)
    total_duration = sum(durations[sites.index(site)] for site in visited_sites)

    # Site data for iteration
    site_data = list(zip(sites, prices, durations))

    # Preference 1: Geographic proximity, starting with AT as a mandatory visit
    for site, price, duration in site_data:
        if site in visited_sites:
            continue  # Skip if already included in the plan
        if total_cost + price > budget or total_duration + duration > hours:
            continue  # Skip if adding this site exceeds budget or time

        # Check for geographic proximity
        site_index = sites.index(site)
        add_site = False
        for added_site in visited_sites:
            added_index = sites.index(added_site)
            if distances[site_index][added_index] <= 1.0:  # Within 1 km
                add_site = True
                break

        if add_site:
            visited_sites.add(site)
            total_cost += price
            total_duration += duration

    return visited_sites, total_cost, total_duration

# Example call to the function with data and distances as previously defined
combined_list_visit, combined_cost, combined_time = optimize_preferences_1_and_4(
    75, 14, prices, durations, sites, distances_km)

print("Recommended Sites:", combined_list_visit)
print("Total Cost:", combined_cost)
print("Total Time:", combined_time)

Recommended Sites: {'AC', 'AT'}
Total Cost: 10.5
Total Time: 2.5


### (e) If Mr. Doe wishes, at the same time, to take into account Preference 2 and Preference 5, which list(s) would you recommend to him?

In [15]:
def optimize_preferences_2_and_5(budget, hours, prices, durations, sites, distances):
    # Start with mandatory sites from Preference 2
    mandatory_sites = {'TE', 'CA'}
    
    visited_sites = set(mandatory_sites)
    total_cost = sum(prices[sites.index(site)] for site in visited_sites)
    total_duration = sum(durations[sites.index(site)] for site in visited_sites)

    # Site data for iteration, adding MO if ML is chosen
    site_data = list(zip(sites, prices, durations))

    for site, price, duration in site_data:
        if site in visited_sites:
            continue  # Skip if already included
        if total_cost + price > budget or total_duration + duration > hours:
            continue  # Skip if adding this site exceeds budget or time

        # Implementing Preference 5: Dependency between ML and MO
        if site == 'ML' and 'MO' not in visited_sites:
            mo_index = sites.index('MO')
            mo_price = prices[mo_index]
            mo_duration = durations[mo_index]

            # Check if adding both ML and MO exceeds budget or time
            if total_cost + price + mo_price <= budget and total_duration + duration + mo_duration <= hours:
                visited_sites.update(['ML', 'MO'])
                total_cost += price + mo_price
                total_duration += duration + mo_duration
        elif site == 'MO' and 'ML' in visited_sites:
            # Add MO if ML is already added
            visited_sites.add('MO')
            total_cost += price
            total_duration += duration
        else:
            # Add other sites normally
            visited_sites.add(site)
            total_cost += price
            total_duration += duration

    return visited_sites, total_cost, total_duration

# Example call to the function with the given data and distances
combined_list_visit, combined_cost, combined_time = optimize_preferences_2_and_5(
    75, 14, prices, durations, sites, distances_km)

print("Recommended Sites:", combined_list_visit)
print("Total Cost:", combined_cost)
print("Total Time:", combined_time)

Recommended Sites: {'TE', 'CA', 'AT', 'ML', 'MO', 'JT'}
Total Cost: 62.0
Total Time: 14.0


### (f) If Mr. Doe wishes, at the same time, to take into account Preference 3 and Preference 4, which list(s) would you recommend to him?

In [17]:
def optimize_preferences_3_and_4(budget, hours, prices, durations, sites, distances):
    # Mandatory site from Preference 4
    mandatory_sites = {'AT'}
    
    visited_sites = set(mandatory_sites)
    at_index = sites.index('AT')  # Get index of Arc de Triomphe
    total_cost = prices[at_index]  # Start with the cost of visiting Arc de Triomphe
    total_duration = durations[at_index]  # Start with the duration of visiting Arc de Triomphe

    # Iterate over the site data
    for site, price, duration in zip(sites, prices, durations):
        if site in visited_sites:
            continue  # Already visited
        if total_cost + price > budget or total_duration + duration > hours:
            continue  # Exceeds budget or time

        # Preference 3: Mutual Exclusivity between AC and SC
        if (site == 'AC' and 'SC' in visited_sites) or (site == 'SC' and 'AC' in visited_sites):
            continue  # Cannot visit both AC and SC

        # Adding site if it's neither AC nor SC involved in mutual exclusivity
        visited_sites.add(site)
        total_cost += price
        total_duration += duration

    return visited_sites, total_cost, total_duration

combined_list_visit, combined_cost, combined_time = optimize_preferences_3_and_4(
    75, 14, prices, durations, sites, distances_km)

print("Recommended Sites:", combined_list_visit)
print("Total Cost:", combined_cost)
print("Total Time:", combined_time)

Recommended Sites: {'TE', 'CA', 'AT', 'ML', 'MO', 'JT'}
Total Cost: 62.0
Total Time: 14.0


### (g) If Mr. Doe wishes, at the same time, to take into account Preference 4 and Preference 5, which list(s) would you recommend to him?

In [18]:
def optimize_preferences_4_and_5(budget, hours, prices, durations, sites, distances):
    # Mandatory site from Preference 4
    mandatory_sites = {'AT'}
    
    visited_sites = set(mandatory_sites)
    at_index = sites.index('AT')  # Get index of Arc de Triomphe
    total_cost = prices[at_index]  # Start with the cost of visiting Arc de Triomphe
    total_duration = durations[at_index]  # Start with the duration of visiting Arc de Triomphe

    # Check if adding ML and MO can fit within the budget and time constraints
    if 'ML' in sites:
        ml_index = sites.index('ML')
        mo_index = sites.index('MO')
        additional_cost = prices[ml_index] + prices[mo_index]
        additional_duration = durations[ml_index] + durations[mo_index]

        if total_cost + additional_cost <= budget and total_duration + additional_duration <= hours:
            # Add both ML and MO to the plan if they fit
            visited_sites.update(['ML', 'MO'])
            total_cost += additional_cost
            total_duration += additional_duration

    # Iterate over remaining sites to see if more can be added without exceeding constraints
    for site, price, duration in zip(sites, prices, durations):
        if site in visited_sites or site in ['ML', 'MO']:
            continue  # Skip if already included or handled by the ML-MO dependency
        if total_cost + price > budget or total_duration + duration > hours:
            continue  # Skip if adding this site exceeds budget or time

        # Add site normally
        visited_sites.add(site)
        total_cost += price
        total_duration += duration

    return visited_sites, total_cost, total_duration

combined_list_visit, combined_cost, combined_time = optimize_preferences_4_and_5(
    75, 14, prices, durations, sites, distances_km)

print("Recommended Sites:", combined_list_visit)
print("Total Cost:", combined_cost)
print("Total Time:", combined_time)

Recommended Sites: {'TE', 'CA', 'AT', 'ML', 'MO', 'JT'}
Total Cost: 62.0
Total Time: 14.0


### (h) If Mr. Doe wishes, at the same time, to take into account Preference 1, Preference 2 and Preference 4, which list(s) would you recommend to him?

In [19]:
def optimize_combined_preferences_1_2_4(budget, hours, prices, durations, sites, distances):
    # Start with mandatory sites from Preferences 2 and 4
    mandatory_sites = {'TE', 'CA', 'AT'}
    
    visited_sites = set(mandatory_sites)
    total_cost = sum(prices[sites.index(site)] for site in mandatory_sites)
    total_duration = sum(durations[sites.index(site)] for site in mandatory_sites)

    # Site data for iteration
    site_data = list(zip(sites, prices, durations))

    for site, price, duration in site_data:
        if site in visited_sites:
            continue  # Skip if already included
        if total_cost + price > budget or total_duration + duration > hours:
            continue  # Skip if adding this site exceeds budget or time

        # Check for geographic proximity for Preference 1
        site_index = sites.index(site)
        add_site = any(distances[site_index][sites.index(vs)] <= 1.0 for vs in visited_sites)

        if add_site:
            visited_sites.add(site)
            total_cost += price
            total_duration += duration

    return visited_sites, total_cost, total_duration

combined_list_visit, combined_cost, combined_time = optimize_combined_preferences_1_2_4(
    75, 14, prices, durations, sites, distances_km)

print("Recommended Sites:", combined_list_visit)
print("Total Cost:", combined_cost)
print("Total Time:", combined_time)

Recommended Sites: {'TE', 'CA', 'AC', 'AT', 'PC'}
Total Cost: 37.0
Total Time: 9.75


### (i) If Mr. Doe wishes, at the same time, to take into account Preference 2, Preference 3 and Preference 5, which list(s) would you recommend to him?

In [20]:
def optimize_preferences_2_3_5(budget, hours, prices, durations, sites, distances):
    # Start with mandatory sites from Preference 2
    mandatory_sites = {'TE', 'CA'}
    
    visited_sites = set(mandatory_sites)
    total_cost = sum(prices[sites.index(site)] for site in mandatory_sites)
    total_duration = sum(durations[sites.index(site)] for site in mandatory_sites)

    # Site data for iteration
    site_data = list(zip(sites, prices, durations))

    for site, price, duration in site_data:
        if site in visited_sites:
            continue  # Skip if already included
        if total_cost + price > budget or total_duration + duration > hours:
            continue  # Skip if adding this site exceeds budget or time

        # Implementing Preference 3: Mutual exclusivity between AC and SC
        if site == 'AC' and 'SC' in visited_sites or site == 'SC' and 'AC' in visited_sites:
            continue
        
        # Implementing Preference 5: Dependency between ML and MO
        if site == 'ML':
            mo_index = sites.index('MO')
            mo_price = prices[mo_index]
            mo_duration = durations[mo_index]
            # Check if adding ML and MO exceeds budget or time
            if total_cost + price + mo_price <= budget and total_duration + duration + mo_duration <= hours:
                visited_sites.update([site, 'MO'])
                total_cost += price + mo_price
                total_duration += duration + mo_duration
        else:
            # Add other sites normally if not ML or conflicting with AC and SC
            visited_sites.add(site)
            total_cost += price
            total_duration += duration

    return visited_sites, total_cost, total_duration

combined_list_visit, combined_cost, combined_time = optimize_preferences_2_3_5(
    75, 14, prices, durations, sites, distances_km)

print("Recommended Sites:", combined_list_visit)
print("Total Cost:", combined_cost)
print("Total Time:", combined_time)

Recommended Sites: {'TE', 'CA', 'AT', 'ML', 'MO', 'JT'}
Total Cost: 62.0
Total Time: 14.0


### (j) If Mr. Doe wishes, at the same time, to take into account Preference 2, Preference 3, Preference 4 and Preference 5, which list(s) would you recommend to him ?

In [21]:
def optimize_complex_preferences(budget, hours, prices, durations, sites, distances):
    # Mandatory sites from Preferences 2 and 4
    mandatory_sites = {'TE', 'CA', 'AT'}
    
    visited_sites = set(mandatory_sites)
    total_cost = sum(prices[sites.index(site)] for site in mandatory_sites)
    total_duration = sum(durations[sites.index(site)] for site in mandatory_sites)

    # Check if adding ML and MO can fit within the budget and time constraints
    if 'ML' in sites:
        ml_index = sites.index('ML')
        mo_index = sites.index('MO')
        if total_cost + prices[ml_index] + prices[mo_index] <= budget and total_duration + durations[ml_index] + durations[mo_index] <= hours:
            visited_sites.update(['ML', 'MO'])
            total_cost += prices[ml_index] + prices[mo_index]
            total_duration += durations[ml_index] + durations[mo_index]

    # Site data for iteration
    site_data = list(zip(sites, prices, durations))

    for site, price, duration in site_data:
        if site in visited_sites:
            continue  # Skip if already included
        if total_cost + price > budget or total_duration + duration > hours:
            continue  # Skip if adding this site exceeds budget or time

        # Implementing Preference 3: Mutual exclusivity between AC and SC
        if (site == 'AC' and 'SC' in visited_sites) or (site == 'SC' and 'AC' in visited_sites):
            continue
        
        # Add other sites normally if not conflicting with AC and SC
        visited_sites.add(site)
        total_cost += price
        total_duration += duration

    return visited_sites, total_cost, total_duration

# Example data setup and function call
sites = ['TE', 'ML', 'AT', 'MO', 'JT', 'CA', 'CP', 'CN', 'BS', 'SC', 'PC', 'TM', 'AC']
prices = [16.5, 14, 10.5, 11, 0, 10, 10, 7, 10, 8.5, 0, 12, 0]
durations = [9/2, 3, 1, 2, 3/2, 2, 5/2, 2, 2, 3/2, 3/4, 2, 3/2]

combined_list_visit, combined_cost, combined_time = optimize_complex_preferences(
    75, 14, prices, durations, sites, distances_km)

print("Recommended Sites:", combined_list_visit)
print("Total Cost:", combined_cost)
print("Total Time:", combined_time)

Recommended Sites: {'TE', 'CA', 'AT', 'ML', 'MO', 'JT'}
Total Cost: 62.0
Total Time: 14.0


### (k) If Mr. Doe wishes, at the same time, to take into account Preference 1, Preference 2, Preference 4 and Preference 5, which list(s) would you recommend to him ?

In [22]:
def optimize_preferences_1_2_4_5(budget, hours, prices, durations, sites, distances):
    # Start with mandatory sites from Preferences 2 and 4
    mandatory_sites = {'TE', 'CA', 'AT'}
    
    visited_sites = set(mandatory_sites)
    total_cost = sum(prices[sites.index(site)] for site in mandatory_sites)
    total_duration = sum(durations[sites.index(site)] for site in mandatory_sites)

    # Check if adding ML and MO can fit within the budget and time constraints
    if 'ML' in sites:
        ml_index = sites.index('ML')
        mo_index = sites.index('MO')
        if total_cost + prices[ml_index] + prices[mo_index] <= budget and total_duration + durations[ml_index] + durations[mo_index] <= hours:
            visited_sites.update(['ML', 'MO'])
            total_cost += prices[ml_index] + prices[mo_index]
            total_duration += durations[ml_index] + durations[mo_index]

    # Site data for iteration, considering geographic proximity
    for site, price, duration in zip(sites, prices, durations):
        if site in visited_sites:
            continue  # Skip if already included
        if total_cost + price > budget or total_duration + duration > hours:
            continue  # Skip if adding this site exceeds budget or time

        # Checking geographic proximity for Preference 1
        site_index = sites.index(site)
        add_site = any(distances[site_index][sites.index(vs)] <= 1.0 for vs in visited_sites)

        if add_site:
            visited_sites.add(site)
            total_cost += price
            total_duration += duration

    return visited_sites, total_cost, total_duration

combined_list_visit, combined_cost, combined_time = optimize_preferences_1_2_4_5(
    75, 14, prices, durations, sites, distances_km)

print("Recommended Sites:", combined_list_visit)
print("Total Cost:", combined_cost)
print("Total Time:", combined_time)

Recommended Sites: {'TE', 'CA', 'AT', 'ML', 'MO', 'JT'}
Total Cost: 62.0
Total Time: 14.0


### (l) If Mr. Doe wishes, at the same time, to take into account Preference 1, Preference 2, Preference 3, Preference 4 and Preference 5, which list(s) would you recommend to him ?

In [23]:
def optimize_all_preferences(budget, hours, prices, durations, sites, distances):
    # Mandatory sites from Preferences 2 and 4
    mandatory_sites = {'TE', 'CA', 'AT'}
    
    visited_sites = set(mandatory_sites)
    total_cost = sum(prices[sites.index(site)] for site in mandatory_sites)
    total_duration = sum(durations[sites.index(site)] for site in mandatory_sites)

    # Attempt to include ML and MO together if ML is selected
    if 'ML' in sites:
        ml_index = sites.index('ML')
        mo_index = sites.index('MO')
        ml_mo_cost = prices[ml_index] + prices[mo_index]
        ml_mo_duration = durations[ml_index] + durations[mo_index]
        
        if total_cost + ml_mo_cost <= budget and total_duration + ml_mo_duration <= hours:
            visited_sites.update(['ML', 'MO'])
            total_cost += ml_mo_cost
            total_duration += ml_mo_duration

    # Iterate over the rest of the sites considering all preferences
    for site, price, duration in zip(sites, prices, durations):
        if site in visited_sites:
            continue  # Already visited
        if total_cost + price > budget or total_duration + duration > hours:
            continue  # Exceeds budget or time

        # Preference 3: Mutual exclusivity between AC and SC
        if (site == 'AC' and 'SC' in visited_sites) or (site == 'SC' and 'AC' in visited_sites):
            continue

        # Check geographic proximity for Preference 1
        site_index = sites.index(site)
        is_close = any(distances[site_index][sites.index(vs)] <= 1.0 for vs in visited_sites)

        if is_close:
            visited_sites.add(site)
            total_cost += price
            total_duration += duration

    return visited_sites, total_cost, total_duration

combined_list_visit, combined_cost, combined_time = optimize_all_preferences(
    75, 14, prices, durations, sites, distances_km)

print("Recommended Sites:", combined_list_visit)
print("Total Cost:", combined_cost)
print("Total Time:", combined_time)

Recommended Sites: {'TE', 'CA', 'AT', 'ML', 'MO', 'JT'}
Total Cost: 62.0
Total Time: 14.0


# Question 3

In [13]:
import scipy.stats as stats

In [15]:
appreciations = [5, 4, 3, 2, 3, 4, 1, 5, 4, 1, 3, 2, 5]

In [20]:
def rank_and_display(sites, values, criterion_name, reverse=False):
    
    ranks = stats.rankdata(values, method='ordinal') # rank the values(high to low)
    sorted_sites = sorted(zip(sites, ranks), key=lambda x: x[1], reverse=reverse)  # sort sites by rank
    
    print(f"\nRanked by {criterion_name} ({'High to Low' if reverse else 'Low to High'}):")
    for site, rank in sorted_sites:
        print(f"{site}: {rank}")

rank_and_display(sites, durations, "Duration")
rank_and_display(sites, appreciations, "Appreciations", reverse=True)  
rank_and_display(sites, prices, "Price") 


Ranked by Duration (Low to High):
PC: 1
AT: 2
JT: 3
SC: 4
AC: 5
MO: 6
CA: 7
CN: 8
BS: 9
TM: 10
CP: 11
ML: 12
TE: 13

Ranked by Appreciations (High to Low):
AC: 13
CN: 12
TE: 11
BS: 10
CA: 9
ML: 8
PC: 7
JT: 6
AT: 5
TM: 4
MO: 3
SC: 2
CP: 1

Ranked by Price (Low to High):
JT: 1
PC: 2
AC: 3
CN: 4
SC: 5
CA: 6
CP: 7
BS: 8
AT: 9
MO: 10
TM: 11
ML: 12
TE: 13


In [22]:
rank_durations = stats.rankdata(durations, method='ordinal')
rank_appreciations = stats.rankdata(appreciations, method='ordinal') 
rank_prices = stats.rankdata(prices, method='ordinal') 

# calculating the Kendall's Tau and Spearman's Rho between these rankings
kendall_tau_duration_appreciation = stats.kendalltau(rank_durations, rank_appreciations)
kendall_tau_duration_prices = stats.kendalltau(rank_durations, rank_prices)
kendall_tau_appreciation_prices = stats.kendalltau(rank_appreciations, rank_prices)

spearman_rho_duration_appreciation = stats.spearmanr(rank_durations, rank_appreciations)
spearman_rho_duration_prices = stats.spearmanr(rank_durations, rank_prices)
spearman_rho_appreciation_prices = stats.spearmanr(rank_appreciations, rank_prices)

print("Kendall's Tau (Duration vs Appreciation):", kendall_tau_duration_appreciation)
print("Kendall's Tau (Duration vs Price):", kendall_tau_duration_prices)
print("Kendall's Tau (Appreciation vs Price):", kendall_tau_appreciation_prices)

print("Spearman's Rho (Duration vs Appreciation):", spearman_rho_duration_appreciation)
print("Spearman's Rho (Duration vs Price):", spearman_rho_duration_prices)
print("Spearman's Rho (Appreciation vs Price):", spearman_rho_appreciation_prices)

Kendall's Tau (Duration vs Appreciation): KendalltauResult(correlation=0.07692307692307691, pvalue=0.7650248459102625)
Kendall's Tau (Duration vs Price): KendalltauResult(correlation=0.564102564102564, pvalue=0.006677354602701825)
Kendall's Tau (Appreciation vs Price): KendalltauResult(correlation=-0.05128205128205127, pvalue=0.8577337519090992)
Spearman's Rho (Duration vs Appreciation): SpearmanrResult(correlation=0.15384615384615383, pvalue=0.6157988686488814)
Spearman's Rho (Duration vs Price): SpearmanrResult(correlation=0.6978021978021978, pvalue=0.008000717411226549)
Spearman's Rho (Appreciation vs Price): SpearmanrResult(correlation=-0.11538461538461539, pvalue=0.7073898735142602)
